In [1]:
import tensorflow as tf
import numpy as np
import os

# Download the dataset (Shakespeare text) for training
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')  # Read and decode text data

# Create a sorted list of unique characters in the text
vocab = sorted(set(text))

# Create a mapping from characters to indices and vice versa
char2idx = {char: idx for idx, char in enumerate(vocab)}  # Dictionary for character-to-index
idx2char = np.array(vocab)  # Array for index-to-character

# Convert the entire text into integer representation using character indices
text_as_int = np.array([char2idx[c] for c in text])


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


array([18, 47, 56, ..., 45,  8,  0])

In [2]:
# Define the length of each sequence for training
seq_length = 100  # Length of each input sequence
examples_per_epoch = len(text) // (seq_length + 1)

# Create a dataset of character indices
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Split the text into sequences of length `seq_length+1`
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Function to create input-target pairs from each sequence
def split_input_target(chunk):
    input_text = chunk[:-1]  # Input: All characters except the last one
    target_text = chunk[1:]  # Target: All characters shifted by one position
    return input_text, target_text

# Map the dataset to get input-target pairs
dataset = sequences.map(split_input_target)


In [3]:
# Define batch size and buffer size for shuffling
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Shuffle the dataset and batch it
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [5]:
# Set model parameters
vocab_size = len(vocab)  # Total number of unique characters
embedding_dim = 256      # Dimension of character embeddings
rnn_units = 1024         # Number of LSTM units

# Build the model architecture
from tensorflow.keras import layers, Sequential

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        # Define input shape explicitly using an Input layer
        layers.Input(batch_shape=(batch_size, None)),  # Include the batch size here

        # Embedding layer for character embeddings
        layers.Embedding(vocab_size, embedding_dim),

        # Stateful LSTM layer
        layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),

        # Dense layer to predict the next character
        layers.Dense(vocab_size)
    ])
    return model

# Instantiate the model with the correct batch size
model = build_model(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

# ... (rest of your code) ...

In [6]:
# Define the loss function
def loss(labels, logits):
    # Sparse categorical crossentropy, since labels are integer indices
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compile the model with Adam optimizer and the custom loss function
model.compile(optimizer='adam', loss=loss)

import os
import tensorflow as tf

# Directory to save checkpoints
checkpoint_dir = './training_checkpoints'
# Ensure the directory exists
os.makedirs(checkpoint_dir, exist_ok=True)

# Checkpoint file naming with '.weights.h5' extension
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

# Define the checkpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)
# Train the model for a set number of epochs
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 18s 69ms/step - loss: 2.8686
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.8520
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.5994
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.4754
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 75ms/step - loss: 1.3976
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 77ms/step - loss: 1.3473
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - loss: 1.3042
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - loss: 1.2670
Epoch 9/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - loss: 1.2361
Epoch 10/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - loss: 1.2013


In [19]:
# ipython-input-18-0f2d0cfb4745
import os
import tensorflow as tf

# ... (rest of the code)

# Directory to save checkpoints
checkpoint_dir = '/content/training_checkpoints'

# Get a list of all checkpoint files in the directory
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith('ckpt')]

# If there are checkpoint files, load the latest one
if checkpoint_files:
    # Find the latest checkpoint file based on epoch number
    latest_checkpoint = max(checkpoint_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
    latest_checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
    print(f"Loading weights from: {latest_checkpoint_path}")  # Print the checkpoint being loaded
    model.load_weights(latest_checkpoint_path) # Load weights from the latest checkpoint path
else:
    print("No checkpoint files found in the directory.")

model.build(tf.TensorShape([1, None]))  # Set input shape to accept variable-length sequences

Loading weights from: /content/training_checkpoints/ckpt_10.weights.h5


In [20]:
# Function to generate text from a trained model
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    # Convert the start string to character indices
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)  # Add batch dimension

    # Placeholder for generated text
    text_generated = []

    # Reset model states before text generation
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)  # Get predictions for the input sequence
        predictions = tf.squeeze(predictions, 0)  # Remove batch dimension

        # Adjust predictions with temperature
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()  # Sample a character index

        # Feed the predicted character as the next input
        input_eval = tf.expand_dims([predicted_id], 0)

        # Convert predicted character index to character and append to generated text
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)  # Return the start string and generated text


In [23]:
# Rebuild the model with batch size of 1 for inference
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# Load the latest checkpoint weights into the model
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))  # Set input shape to accept variable-length sequences


In [24]:
# Generate text using the trained model with a starting string
print(generate_text(model, start_string="ROMEO: "))


AttributeError: 'Sequential' object has no attribute 'reset_states'